In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.3 MB/s 


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving new.xlsx to new.xlsx


In [ ]:
import io
import pandas as pd

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import urllib.request
import math

def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype='uint8')
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)

  return image

DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480
def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  cv2_imshow(img)

In [ ]:
import mediapipe as mp
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
import pandas as pd 

test=pd.read_excel('new.xlsx')

df=pd.DataFrame(
    {
        'product_name':[],
        'src':[],
        'code':[]
    }
)
df.loc[:,'product_name']=test.loc[:,'product_name']
df.loc[:,'src']=test.loc[:,'src']


In [ ]:
for i in range(len(df)):
  try:
    x = df.iloc[i]['src']
    image = url_to_image(x)



    with mp_pose.Pose(
      static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as pose:
      if True:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    # Print nose landmark.
    image_hight, image_width, _ = image.shape
    if not results.pose_landmarks:
      continue
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
    if results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility < 0.5:
      raise Exception()
    if results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility < 0.5:
      raise Exception()
    if results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].visibility < 0.5:
      raise Exception()
    if results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].visibility < 0.5:
      raise Exception()
    if results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].visibility < 0.5:
      raise Exception()
    if results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].visibility < 0.5:
      raise Exception()

    l_s_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * image_width
    l_s_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * image_hight

    r_s_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * image_width
    r_s_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * image_hight

    l_h_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].x * image_width
    l_h_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP].y * image_hight

    r_h_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].x * image_width
    r_h_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP].y * image_hight

    l_k_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x * image_width
    l_k_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y * image_hight

    l_a_x = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x * image_width
    l_a_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y * image_hight


    s_len = math.sqrt(pow(l_s_x - r_s_x, 2) + pow(l_s_y - r_s_y, 2))

    t_len = math.sqrt(pow(l_s_x - l_h_x, 2) + pow(l_s_y - l_h_y, 2))

    h_len = math.sqrt(pow(l_h_x - r_h_x, 2) + pow(l_h_y - r_h_y, 2))

    l_len = math.sqrt(pow(l_k_x - l_h_x, 2) + pow(l_k_y - l_h_y, 2)) + math.sqrt(pow(l_k_x - l_a_x, 2) + pow(l_k_y - l_a_y, 2))

    slen = 1 #어깨 양쪽
    tlen = t_len / s_len #왼쪽어깨 왼쪽 허리
    hlen = h_len / s_len # 허리 양쪽
    llen = l_len / s_len #왼쪽 허리 왼쪽무릎 + 왼쪽 무릎 왼쪽 발목

    code = []

    #다리길이 / 몸통길이

    if  (llen / tlen) < 1.06:
      code += '1'
    elif (llen / tlen) < 1.6:
      code += '2'
    else:
      code += '3'

    #허리길이 / 어깨길이

    if hlen < 0.51:
      code += '1'
    elif hlen < 0.65:
      code += '2'
    else:
      code += '3'

    #몸통길이 / 어깨길이

    if tlen < 1.5:
      code += '1'
    elif tlen < 1.8:
      code += '2'
    else:
      code += '3'


  except:
    code = ""
  df.loc[i,'code']=str(code)
  
df.to_excel('test.xlsx',sheet_name='code')